In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import Adam
import tensorflow as tf
import tensorflow.contrib.slim as slim


from google.colab import drive
drive.mount('/content/drive/')

PATH              = "./drive/Team Drives/final_project_302676838/"
CKPT_DIR          = PATH + 'checkpoint_SID/Sony/'
KERAS_MODEL       = PATH + 'model.h5'

PATCH_SIZE        = 512  # patch size for training

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
def lrelu(x):
    return tf.maximum(x * 0.2, x)

def spacex2(x):
  #import tensorflow as tf
  return tf.depth_to_space(x, block_size=2)
  
def unet(pretrained_weights = None,input_size = (None, None, 4)):
  inputs = Input(input_size) #512x512
    
  conv1 = Conv2D(32, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(inputs)
  conv1 = Conv2D(32, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(conv1) #TODO - UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #256x256

  conv2 = Conv2D(64, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = Conv2D(64, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(conv2)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #128x128

  conv3 = Conv2D(128, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = Conv2D(128, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3) #64x64

  conv4 = Conv2D(256, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4 = Conv2D(256, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(conv4)
  # drop4 = Dropout(0.5)(conv4) #TODO - consider adding dropout
  drop4 = conv4
  pool4 = MaxPooling2D(pool_size=(2, 2))(drop4) #32x32

  conv5 = Conv2D(512, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(pool4) #32x32
  conv5 = Conv2D(512, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(conv5) #32x32
  #drop5 = Dropout(0.5)(conv5)
  drop5 = conv5

  up6 = Conv2D(256, 2, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)  #64x64
  conv6 = Conv2D(256, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(256, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(128, 2, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(128, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(128, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(64, 2, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(64, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(64, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(32, 2, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(32, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = Conv2D(32, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(conv9)
  #conv9 = Conv2D(2, 3, activation = lrelu, padding = 'same', kernel_initializer = 'he_normal')(conv9)

  conv10 = Conv2D(12, 1)(conv9)
  out = Lambda(spacex2 )(conv10) #TODO - do I need this???????
  #out = conv10 

  model = Model(inputs = inputs, outputs = out)
#     model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
  if(pretrained_weights):
    model.load_weights(pretrained_weights)
  return model

In [10]:
########################### tensor flow model
def lrelu(x):
    return tf.maximum(x * 0.2, x)


def upsample_and_concat(x1, x2, output_channels, in_channels):
    pool_size = 2
    deconv_filter = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])

    deconv_output = tf.concat([deconv, x2], 3)
    deconv_output.set_shape([None, None, None, output_channels * 2])

    return deconv_output


def network(input):
    conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1', reuse=tf.AUTO_REUSE)
    conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2', reuse=tf.AUTO_REUSE)
    pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')

    conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1', reuse=tf.AUTO_REUSE)
    conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2', reuse=tf.AUTO_REUSE)
    pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')

    conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1', reuse=tf.AUTO_REUSE)
    conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2', reuse=tf.AUTO_REUSE)
    pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')

    conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1', reuse=tf.AUTO_REUSE)
    conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2', reuse=tf.AUTO_REUSE)
    pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')

    conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1', reuse=tf.AUTO_REUSE)
    conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2', reuse=tf.AUTO_REUSE)

    up6 = upsample_and_concat(conv5, conv4, 256, 512)
    conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1', reuse=tf.AUTO_REUSE)
    conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2', reuse=tf.AUTO_REUSE)

    up7 = upsample_and_concat(conv6, conv3, 128, 256)
    conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1', reuse=tf.AUTO_REUSE)
    conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2', reuse=tf.AUTO_REUSE)

    up8 = upsample_and_concat(conv7, conv2, 64, 128)
    conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1', reuse=tf.AUTO_REUSE)
    conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2', reuse=tf.AUTO_REUSE)

    up9 = upsample_and_concat(conv8, conv1, 32, 64)
    conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1', reuse=tf.AUTO_REUSE)
    conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2', reuse=tf.AUTO_REUSE)

    conv10 = slim.conv2d(conv9, 12, [1, 1], rate=1, activation_fn=None, scope='g_conv10', reuse=tf.AUTO_REUSE)
    out = tf.depth_to_space(conv10, 2)
    return out

TypeError: ignored

In [0]:




sess = tf.Session() 
in_image = tf.placeholder(tf.float32, [None, None, None, 4]) #in_image = tf.placeholder(tf.float32, [1, PATCH_SIZE, PATCH_SIZE, 4])
gt_image = tf.placeholder(tf.float32, [None, None, None, 3]) #gt_image = tf.placeholder(tf.float32, [1, PATCH_SIZE * 2, PATCH_SIZE * 2, 3])
out_image = network(in_image)

saver = tf.train.Saver()
# sess.run(tf.global_variables_initializer())
# G_loss = tf.reduce_mean(tf.abs(out_image - gt_image)) #TODO - try to replace with other network
# lr = tf.placeholder(tf.float32)
# G_opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss, name='train_op')
sess.run(tf.global_variables_initializer())

ckpt = tf.train.get_checkpoint_state(CKPT_DIR)
print('loaded ' + ckpt.model_checkpoint_path)
saver.restore(sess, ckpt.model_checkpoint_path)

#sess.close()

#model = unet()
model = load_model(KERAS_MODEL , custom_objects={'lrelu':lrelu , 'tf':tf}) #load after checkpoint!
#len(model.layers)

In [36]:
index = np.array([[1, 2, 4, 5, 7, 8,  10, 11, 13, 14, 18, 19, 23, 24, 28, 29, 33, 34, 35],
                  [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 21, 23, 26, 28, 31, 33, 36, 38, 40],
                  [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 22, 24, 27, 29, 32, 34, 37, 39, 41]])

for i in range(index.shape[1]):
  #change number here
  L = index[0,i]

  l1 = index[1,i]
  l2 = index[2,i]

  ########
  tensor_w = tf.trainable_variables()[l1].value() 
  tensor_b = tf.trainable_variables()[l2].value()
  w = sess.run(tensor_w)
  b = sess.run(tensor_b)

  print('tf shape:')
  print('weight shape:')
  print(w.shape)
  print('bias shape:')
  print(b.shape)
  print('############')

  t = model.layers[L].get_weights()
  print('unet shape:')
  print('weight shape:')
  print(t[0].shape)
  print('bias shape:')
  print(t[1].shape)
  print('###############')


  print('init weights:')
  print(t[0][:,:,1,1])
  print(t[1][0])
  print('###############')

  t[0] = w
  t[1] = b
  model.layers[L].set_weights(t)

  tt = model.layers[L].get_weights()
  print('trained weights:')
  print(tt[0][:,:,1,1])
  print(tt[1][0])
  print('###############')

  
model.save(KERAS_MODEL)
model.save()

'./drive/Team Drives/final_project_302676838/model.h5'

In [0]:
#change number here
L = 31

l1 = 35

########
tensor_w = tf.trainable_variables()[l1].value() 
w = sess.run(tensor_w)

print('tf shape:')
print('weight shape:')
print(w.shape)
print('############')

t = model.layers[L].get_weights()
print('unet shape:')
print('weight shape:')
print(t[0].shape)
print('###############')

      
print('init weights:')
print(t[0][:,:,1,1])
print(t[1][0])
print('###############')

t[0] = w.swapaxes(2,3)
model.layers[L].set_weights(t)

tt = model.layers[L].get_weights()
print('trained weights:')
print(tt[0][:,:,1,1])
print(tt[1][0])
print('###############')

#model.save(KERAS_MODEL)

In [0]:
a = np.array([[1, 2] , [3 ,4] , [5, 6]])
print(a)
t = a.swapaxes(0,1)
print(t)

[[1 2]
 [3 4]
 [5 6]]
[[1 3 5]
 [2 4 6]]
